In [ ]:
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.image as mpimg
import random
from tensorflow.keras import layers
from tensorflow import keras
from google.colab import drive
from google.colab import files
from keras.preprocessing import image
drive.mount('/content/drive')
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img,img_to_array

NameError: name 'drive' is not defined

In [ ]:
base_dir= '/content/drive/MyDrive/Bangkit_Fruits'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

In [ ]:
# Mempersiapkan train_dataset
train_dataset = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=(150, 150),
    batch_size=32,
    label_mode='categorical',
    seed= 314
    )

# Mempersiapkan validation_dataset
validation_dataset = tf.keras.utils.image_dataset_from_directory(
    validation_dir,
    image_size=(150, 150),
    batch_size=32,
    label_mode='categorical',
    seed= 314
    )

In [ ]:
class_names = ["Apel","Anggur","Buah Naga","Ceri", "Durian", "Jambu Biji","Jeruk", "Kiwi", "Lemon", "Mangga", "Nanas", "Pir","Pisang", "Semangka", "Stroberi" ]

In [ ]:
def random_class_visualization(class_names, directory):
    """Visualizes random images from specified classes.

    Args:
        class_names: List of class names (subdirectory names in `directory`).
        directory: Path to the directory containing class subdirectories.

    This function randomly selects images from the given class directories and
    displays them in a 2x5 grid with their class names as titles.
    """
    plt.figure(figsize=(10, 3))
    for i in range(10):
        random_class = random.choice(class_names)
        class_path = os.path.join(directory, random_class)
        images = os.listdir(class_path)
        random_image = images[random.randint(0, len(images) - 1)]
        image_path = os.path.join(class_path, random_image)

        plt.subplot(2, 5, i+1)
        img = mpimg.imread(image_path)
        plt.imshow(img)
        plt.title(random_class)
        plt.axis('off')

    plt.tight_layout()
    plt.show()

random_class_visualization(class_names, validation_dir)

In [ ]:
# Check Shape per batch
for images, labels in train_dataset.take(1):  # Iterate through the first batch in the training dataset
    print('Batch Image Shape:', images.shape)
    print('Batch Labels Shape:', labels.shape)
    break

In [ ]:
rescale_layer = tf.keras.Sequential([
    keras.layers.Rescaling(1./255)
])

augmentation_layer = tf.keras.Sequential([
    keras.layers.RandomFlip("horizontal"),
    keras.layers.RandomRotation(
        (-0.2, 0.2),
        fill_mode='reflect',
        interpolation='bilinear'
    ),
    keras.layers.RandomTranslation(
        (-0.15, 0.15),
        (-0.15, 0.15),
        fill_mode='reflect',
        interpolation='bilinear'
    ),
])

def preprocess_dataset(dataset, augment=False):
    # Rescale all datasets
    dataset = dataset.map(lambda x, y: (rescale_layer(x), y))

    # Data augmentation only on the training set.
    if augment:
        dataset = dataset.map(lambda x, y: (augmentation_layer(x, training=True), y))

    return dataset

train_ds = preprocess_dataset(train_dataset, augment=True)
val_ds = preprocess_dataset(validation_dataset)

In [ ]:
# Inisialisasi base model yang akan kita gunakan
input_shape = (150, 150, 3)
base_model = tf.keras.applications.efficientnet_v2.EfficientNetV2S(
    include_top=False,
    weights='imagenet',
    input_shape=input_shape,
    include_preprocessing=False
)
base_model.trainable = False

In [ ]:
# Functional API for Keras Model Building
inputs = tf.keras.Input(shape=input_shape)
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(640, activation='relu')(x)
x = layers.Dropout(0.6)(x)
x = layers.Dense(320, activation='relu')(x)
x = layers.Dropout(0.2)(x)
outputs = layers.Dense(15, activation='softmax')(x)

# Create the final model using Functional API
model = tf.keras.Model(inputs, outputs)

# Display the summary of the final model
model.summary()

In [ ]:
# Mengatur parameter pelatihan
learning_rate = 1e-4
optimizer = tf.keras.optimizers.Adam(learning_rate= learning_rate)
model.compile(
    optimizer= optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy']
    )

In [ ]:
class EarlyStoppingCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if logs['accuracy']>0.85:
            self.model.stop_training = True
            print("\nReached 95% accuracy so cancelling training!")
callback = EarlyStoppingCallback()

In [ ]:
EPOCHS = 20

# Pelatihan Model
history = model.fit(
    train_ds,
    validation_data = val_ds,
    epochs = EPOCHS,
    verbose = 1,
    callbacks=[callback]
    )

In [ ]:
def plot_loss_acc(history):
    '''Plots the training and validation loss and accuracy from a history object'''
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(len(acc))

    fig, ax = plt.subplots(1,2, figsize=(12, 6))
    ax[0].plot(epochs, acc, 'bo', label='Training accuracy')
    ax[0].plot(epochs, val_acc, 'b', label='Validation accuracy')
    ax[0].set_title('Training and validation accuracy')
    ax[0].set_xlabel('epochs')
    ax[0].set_ylabel('accuracy')
    ax[0].legend()

    ax[1].plot(epochs, loss, 'bo', label='Training Loss')
    ax[1].plot(epochs, val_loss, 'b', label='Validation Loss')
    ax[1].set_title('Training and validation loss')
    ax[1].set_xlabel('epochs')
    ax[1].set_ylabel('loss')
    ax[1].legend()

    plt.show()

plot_loss_acc(history)

In [ ]:
uploaded = files.upload()

for fn in uploaded.keys():
    path = fn
    # Load gambar dan ubah ukurannya
    img = image.load_img(path, target_size=(150, 150))
    plt.imshow(img)

    # Preprocessing gambar
    x = image.img_to_array(img) / 255.0
    x = np.expand_dims(x, axis=0)

    # Prediksi kelas
    classes = model.predict(x)  # or model.predict(x)
    class_index = np.argmax(classes[0])

    # Menampilkan prediksi
    plt.title(f"Predicted Label: {class_names[class_index]}")
    plt.axis("off")
    plt.show()

In [ ]:
# Save the model
SAVED_MODEL = "fruits_saved_model"
tf.saved_model.save(model, SAVED_MODEL)

In [ ]:
model.save('/content/drive/MyDrive/final.h5')

In [ ]:
tflite_model = tf.keras.models.load_model("/content/drive/MyDrive/final.h5")
converter = tf.lite.TFLiteConverter.from_keras_model(tflite_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_save = converter.convert()
open("/content/drive/MyDrive/model.tflite", "wb").write(tflite_save)